# Processing images and labels into chips

Planet imagery was originally processed into larger tiles of 2368 x 2358 pixels at a resolution of 0.000025$^\circ$. Labelling was undertaken on only a subset of each tile, corresponding to a 0.005$^\circ$ target (~550 m). For release, the imagery was cropped to the target box and resampled to make chips of 224x224 pixels, and labels were rasterized to the same dimensions. 

In [1]:
import os
from pathlib import Path
import rioxarray as rxr
from rasterio.enums import Resampling
import geopandas as gpd
from shapely.geometry import Polygon, box
import pandas as pd
import numpy as np
from datetime import datetime as dt

## Setup

In [2]:
root_dir = os.environ["HOME"]
proj_dir = Path(root_dir) / "projects/lacunalabels"
data_dir = Path(root_dir) / "data"
chip_dir = Path(data_dir) / "lacuna/images"
label_dir = Path(data_dir) / "lacuna/labels"
image_dir = Path(os.path.dirname(root_dir)) / "data/imagery/planet/tiles"
log_path = Path(root_dir) / "logs/image-chipping.log"

for d in [chip_dir, label_dir]:
    if not os.path.isdir(d):
        os.makedirs(d)

### Functions

In [3]:
def target_poly(x, y, w=0.0025, crs="epsg:4326"):
    poly = box(x-w, y-w, x+w, y+w)
    poly_gdf = gpd.GeoDataFrame({"geometry": [poly]}, crs=crs)
    return poly_gdf

def chipper(image, x, y, rows, cols, dst_path, log, decimals=4, overwrite=True, 
            resample_method=None): 
    
    if not resample_method:
        resample_method=Resampling.cubic 
        
    if not overwrite and os.path.exists(dst_path):
        msg = f"{os.path.basename(dst_path)} exists, skipping"
        print(msg, file=log, flush=True)
    
    else: 
                
        bnds = target_poly(x, y)
        chip = (
            image.rio.clip(bnds.geometry)
            .rio.reproject(image.rio.crs, shape=(rows, cols), 
                           resampling=resample_method)
        )

        # checks
        chip_bnds = chip.rio.bounds()
        try:
            assert np.isclose(np.array(chip_bnds), np.array(bnds.bounds)[0]).all()
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} has incorrect bounds"
            print(msg, file=log, flush=True)
            raise err
        try:    
            assert chip.shape[1:3] == (rows, cols)
        except AssertionError as err:
            msg = f"{dt.now()}: {os.path.basename(dst_path)} incorrect output shape"
            print(msg, file=log, flush=True)
            raise err
    
        chip.rio.to_raster(dst_path)   
        msg = f"{dt.now()}: created {os.path.basename(dst_path)}"
        print(msg, file=log, flush=True)
    
    return msg


### Catalogs

In [9]:
catalog = pd.read_csv(Path(proj_dir) /\
                      "data/interim/assignments_full_wtiles.csv")
chip_catalog = (
    catalog[["name", "image_date", "x", "y", "destfile"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

## Image chipping

Processing image chips into 224x224. 

In [88]:
# qs = catalog.query("Type=='Q'").sample(n=50,random_state=1).name

In [10]:
log = open(log_path, "a+")
print(f"\nStarting at {dt.now()}\n", file=log, flush=True)

chip_list = []
# for i, row in chip_catalog.query("name in @qs").iterrows():
for i, row in chip_catalog.iterrows():
    
    chip_name = f"{row['name']}-{row['image_date']}.tif"
    chip_path = str(Path(chip_dir) / chip_name)
    
    row["chip"] = chip_name
    chip_list.append(row)
    result = chipper(
        rxr.open_rasterio(Path(image_dir) / row.destfile), 
        row.x, row.y, 224, 224, chip_path, log, 4, False
    )
print(f"\nFinished at {dt.now()}", file=log, flush=True)
log.close()

Combine results back to catalog with chip name

In [43]:
chip_catalogf = pd.concat([pd.DataFrame([l.to_list()], columns=l.index) 
                           for l in chip_list])
chip_catalogf.reset_index(drop=True, inplace=True)
chip_catalogf.drop(columns=["destfile", "x", "y"], inplace=True)
chip_catalogf.loc[0:2]

,name,image_date,chip
0,ET0007182,2017-08-15,ET0007182-2017-08-15.tif
1,NE3372442,2021-08-15,NE3372442-2021-08-15.tif
2,SN0105655,2020-02-15,SN0105655-2020-02-15.tif


In [48]:
catalog2 = pd.merge(catalog, chip_catalogf, how="left")
catalog2.to_csv(
    Path(proj_dir) / "data/interim/label_catalog_int.csv", index=False
)